In [78]:
# Испорт модулей

from bs4 import BeautifulSoup #модуль для парсинга
# Модули selenium используются для взаимодействия с сайтом во время парсинга (Нажатие кнопки, например)
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from nltk.tokenize import word_tokenize
from nltk import pos_tag, download
from nltk.stem import WordNetLemmatizer
from nltk import SnowballStemmer
import pandas as pd
import time 
import re
import pandas
import datetime
import requests
from collections import Counter

In [9]:
mounts = [
    "января",
    "февраля",
    "марта",
    "апреля",
    "мая",
    "июня",
    "июля",
    "августа",
    "сентября",
    "октября",
    "ноября",
    "декабря"
]

In [48]:
# И текстового формата даты с сайта получает datetime
def parseDateTime(past_new):
    if "час" in past_new:
        return datetime.date.today()
    elif past_new == "1 день назад":
        return datetime.date.today() - datetime.timedelta(days=1)
    elif past_new == "2 дня назад":
        return datetime.date.today() - datetime.timedelta(days=2)
    else:
        year = int(past_new.split(" ")[2]) if len(past_new.split(" ")) >= 3 else datetime.date.today().year
        mount = mounts.index(past_new.split(" ")[1]) + 1
        day = int(past_new.split(" ")[0])
        return datetime.date(year, mount, day)
    

In [61]:
#Функция парсинга одной страницы новостей сайта mk.ru

def get_news_mk(
        topic: str, #Тема новостей 
        start_period: datetime, #Начало периода
        end_period: datetime, #Конец периода
        page: int, #Номер страницы
    ):
    
    #Формируем ссылку для парсинга
    url = f'https://www.mk.ru/{topic}/{page}/'
    #Отправляем GET запрос по ссылке
    r = requests.get(url)
    #Парсим полученный HTML
    soup = BeautifulSoup(r.text, 'html.parser')
    
    #Находим все теги h2 с указанным классом (Там написана дата новостей)
    all_dates = soup.find_all('h2', {"class": "article-listing__subtitle"})
    
    #Если новостей больше не осталось, выходим из цикла
    if len(all_dates) == 0:
        return False
    
    #Проходимся по всем найденным датам
    for date_tag in all_dates:
        #Извлекаем из полученного текста год, месяц и день
        year = int(date_tag.text.split(" ")[2])
        mount = mounts.index(date_tag.text.split(" ")[1]) + 1
        day = int(date_tag.text.split(" ")[0])
        
        #Формируем дату формата datetime.date()
        date = datetime.date(year, mount, day)

        #Проверяем, вхдит ли дата в нужный период 
        if (date <= end_period):
            if (date >= start_period):
                
                #Этот кусок кода выполняется, если дата входит в период 
                
                #Проходим по всем новостям этой даты и записываем заголовки 
                with open('mknews.txt', 'w') as f:
                    for new in date_tag.parent.find_all('li', {"class": "article-listing__item"}):
                        line = new.find('h3', {"class": "listing-preview__title"}).text
                        mk_words.append(line)
            else: #Если дата вышла из периода, выходим из цикла
                return False
    return True

In [70]:
#Функция парсингда новостей с сайта news.mail.ru

def get_news_mail(
        topic: str, #Тема новостей 
        start_period: datetime, #Начало периода
        end_period: datetime, #Конец периода
    ):

    # Созадние объекта свойств Хрома
    options = webdriver.ChromeOptions()

    # Определение свойств Хрома
    options.add_experimental_option('useAutomationExtension', False)
    options.add_experimental_option('excludeSwitches',['enable-automation'])

    # Созадние объекта запуска Хрома
    # на МАС и на Linux предварительно нужно  скачать chromedriver, убедиться, что версия chromedriver 
    # не конфликтует с версией chrom-а и в папке chromedriver выполнить команду 
    #"xattr -d com.apple.quarantine chromedriver"
    #Также тут нужно указать полный путь к файлу
    browser = webdriver.Chrome('/Users/marykhalatyan/my_projects/news_parser/chromedriver', options = options)
    
    # Заход на сайт
    browser.get(f'https://news.mail.ru/{topic}/')


    # Извлекаем HTML
    soup = BeautifulSoup(browser.page_source)
    
    # Тут нажимаем кнопку "Показать еще", пока дойдем до конца указанного периода
    while True:
        # Извлекаем дату последней новости загруженной страницы 
        past_new = soup.find_all(
            'div', {"class": "newsitem newsitem_height_fixed js-ago-wrapper js-pgng_item"}
        )[-1].find('span', {"class": "newsitem__param js-ago"}).text
        
        
        date = parseDateTime(past_new)
            
        
        if (date <= end_period):
            if (date >= start_period): 
                # Если период не закончился, нажимаем кнопку
                element = browser.find_element_by_css_selector('.button.margin_top_20.js-pgng_more_link')
                
                if element:
                    element.click()
                    soup = BeautifulSoup(browser.page_source)
                else: #Если больше нет кнопки "Показать еще", выходим из цикла
                    break
            else:
                break
                
    # С загруженной страницы извлекаем заголовки и записываем в файле
    with open('mailnews.txt', 'w') as f:
        
        for new in soup.find_all('div', {"class": "newsitem newsitem_height_fixed js-ago-wrapper js-pgng_item"}):
            past_new = new.find('span', {"class": "newsitem__param js-ago"}).text
            date = parseDateTime(past_new)

            if (date <= end_period) and (date >= start_period): 
                line = new.find('a', {"class": "newsitem__title link-holder"}).text
                
                f.write(line)
                f.write('\n')

In [67]:
# Вызываем функцию парсинга сайта mk.ru
mk_words = []
page = 1
while True:
    # Если есть еще страницы с новостями данного периода, вызываем еще раз
    if not get_news_mk(
        'economics',
        datetime.date.today() - datetime.timedelta(days=30) ,
        datetime.date.today(),
        page
    ): 
        break
    page += 1

In [68]:
# записываем заголовки с сайта в файл
with open('mknews.txt', 'w') as f:
    for line in mk_words:
        f.write(line)
        f.write('\n')

In [71]:
# Вызываем функцию парсинга сайта news.mail.ru
get_news_mail(
    'economics',
    datetime.date.today() - datetime.timedelta(days=30) ,
    datetime.date.today(),
)

In [87]:
#Эта часть кода была взята из примеров, адаптирована под мой код и записаны комментарии

# Цикл будет проходить по файлам, где записаны заголовки 
files = ['mailnews.txt', 'mknews.txt']
for file in files:
    news = []
    #Читает все заголовки и добавляет их в список
    with open(file, 'r') as f:
        for line in f:
            news.append(line)
    # Получает все слова из приложений
    text = ' '.join(news).lower()
    tokens = []
    #Идет проверка слов на корректность
    for token in word_tokenize(text):
        try:
            #Если это не слово, а какой-то символ, в список оно не добавляется
            re.match('\W+|\d+', token).group(0)
        except:
            #Если все хорошо, добавляем в список 
            tokens.append(token)
    
    #Тут нужно опять пройтись по словам и привести их в лемматизированный вид
    res = []
    for element in pos_tag(tokens, lang = 'rus'):
        #Из всех слов берет только существительные и глаголы (Тут можешь подсказать, что еще можно взять)
        if element[1] == 'S' or element[1] == 'V':
            #Выбранные слова определяет как русские и лемматизирует
            res.append(SnowballStemmer('russian').stem(WordNetLemmatizer().lemmatize(element[0], pos = 'n')))
    
    #Записывает слова и их количества (В сортированном виде) в xlsx фалы
    #Counter - считает количество слов 
    #most_common сортирует
    pd.DataFrame(dict(Counter(res).most_common()).items()).to_excel('files/Мешок_слов_' + file + '.xlsx')